In [2]:
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.decomposition import TruncatedSVD
from scipy.linalg import svd

In [9]:
df = pd.read_csv('../tugas2/data_train_vsm.csv')
df

,Kategori Berita,abad,abai,abbas,abc,abdel,abidjan,abu,acara,acerbi,...,yoon,yordania,york,young,youtube,yubileum,yudea,yudhoyono,zionis,zona
0,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
1,nasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
2,nasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
3,nasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.205246,0.000000,0.0
4,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.088283,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
76,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
77,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.037868,0.0,0.0,0.0,0.000000,0.000000,0.0
78,internasional,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0


In [10]:
df.drop(['Kategori Berita'], axis=1, inplace=True)

In [11]:
csr_mat = df.values

In [6]:
U, s, VT = svd(csr_mat)

In [15]:
VT.shape

(2399, 2399)

In [7]:
Sigma = np.zeros((csr_mat.shape[0], csr_mat.shape[1]))

min_dim = min(csr_mat.shape[0], csr_mat.shape[1])
Sigma[:min_dim, :min_dim] = np.diag(s)

In [ ]:
n_elements = 2
Sigma = Sigma[:, :n_elements]
VT = VT[:n_elements, :]

T = U[:, :n_elements] @ np.diag(s[:n_elements])



In [ ]:
svd = TruncatedSVD(n_components=1799)
svd.fit(csr_mat)
result = svd.transform(csr_mat)
print(result.shape)

(80, 10)


In [ ]:
query = csr_mat[0, :]
query_reduced = query @ VT.T

[0. 0. 0. ... 0. 0. 0.]


In [12]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


# Calculate cosine similarity (using reduced document and query vectors)
similarities = [cosine_similarity(query_reduced, T[i, :]) for i in range(3)] # Calculate for first 3 documents

# Rank documents
ranking = sorted([(similarities[i], f"Doc{i+1}") for i in range(3)], reverse=True)


print("Document Ranking:")
for similarity, doc_name in ranking:
    print(f"{doc_name}: {similarity}")

Document Ranking:
Doc1: 1.0
Doc3: 0.5955423225823545
Doc2: 0.10567980220017449


In [24]:
oc1_approx = T[0, :] @ VT  # Project back to the original space
oc1_approx

array([0.00186302, 0.00289307, 0.00339565, ..., 0.00072149, 0.02703836,
       0.00253726])